In [9]:
%pip install -e "./Pitmodule"
from Pitmodule.pitframe import *
from Pitmodule.population import PopulationFrame,default_tables
import pandas as pd
# zmienna default tables z modulu population przechowuje wybrane tabele
default_tables
populacje2020 = PopulationFrame()
lud_gminy = populacje2020.data["Gminy"]
lud_gminy

Obtaining file:///C:/Users/adomoslaws001/nypd-finalassignment/PitmoduleNote: you may need to restart the kernel to use updated packages.

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: Pitmodule
    Found existing installation: Pitmodule 1.0
    Uninstalling Pitmodule-1.0:
      Successfully uninstalled Pitmodule-1.0
  Running setup.py develop for Pitmodule


,,populacja
id,jst,
0201011,Bolesławiec,29284
0201022,Bolesławiec,10387
0201032,Gromadka,3841
0201043,Nowogrodziec,10631
0201052,Osiecznica,5036
...,...,...
3218053,Węgorzyno,4923
3261011,Koszalin,80478
3262011,Szczecin,300952


In [5]:
populacje2020.data["Wojewodztwa"]

,,populacja
id,jst,
1054.0,96,0
10549.0,85,0
1073478.0,WiekmobilnyMobilityage\n18-44latamężczyźniikobiety\n18-44yearsmalesandfemales\n\n,0
111551.0,3-6lat,0
11674.0,84,0
...,...,...
91998.0,70-74,0
9244.0,90-94,0
98736.0,50-54,0


In [36]:
empty_df = pd.DataFrame()

age_cond = ['27','28','29',
                '30-34',
                '35-39',
                '40-44',
                '45-49',
                '50-54',
                '55-59',
                '60-64',
                '65-69',
                '70-74',
                '75-79',
                '80-84',
                '75iwięcej',
                '80iwięcej',
                '85iwięcej'
            ]


with pd.ExcelFile("./data/ludnosc_stan_struktura/tabela06.xls") as xls:
    empty_df = empty_df.append(pd.read_excel(xls,sheet_name="Podkarpackie",skiprows=7,usecols="A:C"))

m={empty_df.columns[0]:"jst",
        empty_df.columns[1]: "id",
        empty_df.columns[2]: "ludnosc"}

empty_df = empty_df.rename(columns= m)
empty_df['id'] = empty_df['id'].astype('str')
empty_df.iloc[:,1] = empty_df.iloc[:,1].str.replace(' ','')
empty_df.iloc[:,0] = empty_df.iloc[:,0].str.replace(' ','')

empty_df = empty_df[ empty_df.iloc[:,1].apply(lambda x: len(str(x)) > 0 and not pd.isnull(x)) | empty_df.iloc[:,0].isin(age_cond)]

empty_df.reset_index(drop=True,inplace=True)

empty_df.loc[empty_df['id'].apply(lambda x: len(x) == 7),'ludnosc'] = 0 
empty_df = empty_df[ empty_df.iloc[:,1].apply(lambda x: len(str(x)) > 0 and not pd.isnull(x)) | empty_df.iloc[:,0].isin(age_cond)]

empty_df.reset_index(drop=True,inplace=True)
val = empty_df.iloc[0,1]
name =empty_df. iloc[0,0]
df_dict = empty_df.to_dict(orient='records')
for row in df_dict:
    if row['id'] == '' or pd.isnull(row['id']):
        row['id'] = val
        row['jst'] = name
    else:
        val = row['id']
        row['ludnosc'] = 0
        name = row['jst']
empty_df = pd.DataFrame(df_dict)
empty_df['ludnosc'] = empty_df['ludnosc'].astype(int)
empty_df  = empty_df.groupby(["id","jst"],as_index=False)["ludnosc"].sum()
empty_df = empty_df[empty_df['id'].notna() & empty_df['ludnosc'] > 0 ]

empty_df

,id,jst,ludnosc


In [37]:
print(empty_df)

IndexError: single positional indexer is out-of-bounds

In [6]:
age_cond = ['27','28','29',
                '30-34',
                '35-39',
                '40-44',
                '45-49',
                '50-54',
                '55-59',
                '60-64',
                '65-69',
                '70-74',
                '75-79',
                '80-84',
                '75iwięcej',
                '80iwięcej',
                '85iwięcej'
            ]

tabs = default_tables = {
    "Gminy":"tabela12.xls",
    "Powiaty":"tabela06.xls",
    "Wojewodztwa":"tabela03.xls"
}
jst = "Gminy"
jst_frame = pd.DataFrame()
with pd.ExcelFile("./data/ludnosc_stan_struktura/"+tabs[jst]) as xls:
    with pd.ExcelWriter('output.xlsx') as wrt:
        for sheet in xls.sheet_names:
            tmp =pd.read_excel(xls,sheet_name=sheet,skiprows =7,usecols="A:C")
            
            # rename columns
            m={tmp.columns[0]:"jst",
            tmp.columns[1]: "id",
            tmp.columns[2]: "populacja"}
            tmp = tmp.rename(columns= m)

            tmp = tmp.astype('str')
            tmp.iloc[:,1] = tmp.iloc[:,1].str.replace(' ','')
            tmp.iloc[:,0] = tmp.iloc[:,0].str.replace(' ','')

            #slicing & editing
            tmp =  tmp[tmp.iloc[:,1].apply(lambda x: len(str(x)) > 0 and not pd.isnull(x)) | tmp.iloc[:,0].isin(age_cond)]
            
            #preapare data for swaping values
            code_len = {'Gminy':7,'Powiaty':4,'Wojewodztwa':2}
            tmp.loc[tmp['id'].apply(lambda x: len(x) == code_len[jst]),'populacja'] = 0 

            val = tmp.iloc[0,1]
            name =tmp. iloc[0,0]
            df_dict = tmp.to_dict(orient='records')
            for row in df_dict:
                if row['id'] == '' or pd.isnull(row['id']):
                    row['id'] = val
                    row['jst'] = name
                else:
                    val = row['id']
                    row['populacja'] = 0
                    name = row['jst']
            tmp = pd.DataFrame(df_dict)
            tmp  = tmp.groupby(by=['id','jst'])[['populacja']].sum()
            tmp.to_excel(wrt,sheet_name= sheet) 
            jst_frame = jst_frame.append(tmp)
        
jst_frame

C:\Users\ADOMOS~1\AppData\Local\Temp/ipykernel_22520/4275640433.py:59: FutureWarning: Dropping invalid columns in DataFrameGroupBy.add is deprecated. In a future version, a TypeError will be raised. Before calling .add, select only columns which should be valid for the function.
  tmp  = tmp.groupby(by=['id','jst'])[['populacja']].sum()


,
id,jst
0201011,Bolesławiec
0201022,Bolesławiec
0201032,Gromadka
0201043,Nowogrodziec
0201052,Osiecznica
...,...
3218053,Węgorzyno
3261011,Koszalin
3262011,Szczecin


In [18]:
pd.ExcelFile("./data/ludnosc_stan_struktura/tabela12.xls").sheet_names 

['Dolnośląskie',
 'Kujawsko-pomorskie',
 'Lubelskie',
 'Lubuskie',
 'Łódzkie',
 'Małopolskie',
 'Mazowieckie',
 'Opolskie',
 'Podkarpackie',
 'Podlaskie',
 'Pomorskie',
 'Śląske',
 'Świętokrzyskie',
 'Warmińsko-mazurskie',
 'Wielkopolskie',
 'Zachodniopomorskie']

In [6]:
empty_df

,jst,id,ludnosc
0,JeleniaGóra,0261011,0
1,JeleniaGóra,0261011,800
2,JeleniaGóra,0261011,784
3,JeleniaGóra,0261011,828
4,JeleniaGóra,0261011,5024
...,...,...,...
2771,Żukowice,0203062,218
2772,Żukowice,0203062,139
2773,Żukowice,0203062,65
2774,Żukowice,0203062,47


In [91]:
non_empty = empty_df.loc[empty_df['id'].apply(lambda x: len(str(x)) > 0 and not pd.isnull(x))] 
non_empty

,jst,id,ludnosc
0,JeleniaGóra,0261011,78335
16,Legnica,0262011,98436
32,Wrocław,0264011,641928
48,Wrocław-Fabryczna,0264029,207715
64,Wrocław-Krzyki,0264039,179891
...,...,...,...
2699,ZłotyStok,0224073,4410
2715,Żarów,0219083,12335
2731,Żmigród,0220063,14608
2747,Żórawina,0223092,11406


In [74]:
val = empty_df.iloc[0,1]
sum = 0
for index in empty_df.index:
    if empty_df.iloc[index,1] == '' or pd.isnull(empty_df.iloc[index,1]):
        empty_df.iloc[index,1] = val
        sum += int(empty_df.iloc[index,2])
    else:
        val = empty_df.iloc[index,1]
        sum = 0 

empty_df

,jst,id,ludnosc
0,JeleniaGóra,0261011,78335
1,27,0261011,800
2,28,0261011,784
3,29,0261011,828
4,30-34,0261011,5024
...,...,...,...
2774,65-69,0203062,218
2775,70-74,0203062,139
2776,75-79,0203062,65
2777,80-84,0203062,47


,jst,id,ludnosc
0,JeleniaGóra,0261011,78335
1,27,,800
2,28,,784
3,29,,828
4,30-34,,5024
...,...,...,...
2774,65-69,,218
2775,70-74,,139
2776,75-79,,65
2777,80-84,,47


In [71]:
val = empty_df.iloc[0,1]
val_set = [val]
for index in empty_df.index:
    cell = empty_df.iloc[index,1]
    if not (cell == '' or pd.isnull(cell)):
        val = cell
        val_set.append(val)
        
    
len(val_set)

175